# Multi-agent recruiting workflow 
Last tested with letta version `0.5.3`

## Section 0: Setup a MemGPT client 

In [1]:
from letta import create_client 

client = create_client() 

Initializing database...


In [2]:
from letta import LLMConfig, EmbeddingConfig

client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini")) 
client.set_default_embedding_config(EmbeddingConfig.default_config("text-embedding-ada-002")) 

## Section 1: Shared Memory Block 
Each agent will have both its own memory, and shared memory. The shared memory will contain information about the organization that the agents are all a part of. If one agent updates this memory, the changes will be propaged to the memory of all the other agents. 

In [3]:
from letta.schemas.block import Block 

org_description = "The company is called AgentOS " \
+ "and is building AI tools to make it easier to create " \
+ "and deploy LLM agents."

org_block = Block(label="company", value=org_description )

In [4]:
org_block

Block(value='The company is called AgentOS and is building AI tools to make it easier to create and deploy LLM agents.', limit=2000, template_name=None, template=False, label='company', description=None, metadata_={}, user_id=None, id='block-6db0fe1a-1f5e-44ab-852c-a2df8d7ab80e')

In [5]:
from letta.schemas.memory import BasicBlockMemory

class OrgMemory(BasicBlockMemory): 

    def __init__(self, persona: str, org_block: Block): 
        persona_block = Block(label="persona", value=persona)
        super().__init__(blocks=[persona_block, org_block])
        

## Section 2: Orchestrating Multiple Agents 
We'll implement a recruiting workflow that involves evaluating an candidate, then if the candidate is a good fit, writing a personalized email on the human's behalf. Since this task involves multiple stages, sometimes breaking the task down to multiple agents can improve performance (though this is not always the case). We will break down the task into: 

1. `eval_agent`: This agent is responsible for evaluating candidates based on their resume
2. `outreach_agent`: This agent is responsible for writing emails to strong candidates
3. `recruiter_agent`: This agent is responsible for generating leads from a database 

Much like humans, these agents will communicate by sending each other messages. We can do this by giving agents that need to communicate with other agents access to a tool that allows them to message other agents. 

#### Evaluator Agent
This agent will have tools to: 
* Read a resume 
* Submit a candidate for outreach (which sends the candidate information to the `outreach_agent`)

In [6]:
def read_resume(self, name: str): 
    """
    Read the resume data for a candidate given the name

    Args: 
        name (str): Candidate name 

    Returns: 
        resume_data (str): Candidate's resume data 
    """
    import os
    filepath = os.path.join("data", "resumes", name.lower().replace(" ", "_") + ".txt")
    #print("read", filepath)
    return open(filepath).read()

def submit_evaluation(self, candidate_name: str, reach_out: bool, resume: str, justification: str): 
    """
    Submit a candidate for outreach. 

    Args: 
        candidate_name (str): The name of the candidate
        reach_out (bool): Whether to reach out to the candidate
        resume (str): The text representation of the candidate's resume 
        justification (str): Justification for reaching out or not
    """
    from letta import create_client 
    client = create_client()
    message = "Reach out to the following candidate. " \
    + f"Name: {candidate_name}\n" \
    + f"Resume Data: {resume}\n" \
    + f"Justification: {justification}"
    # NOTE: we will define this agent later 
    if reach_out:
        response = client.send_message(
            agent_name="outreach_agent", 
            role="user", 
            message=message
        ) 
    else: 
        print(f"Candidate {candidate_name} is rejected: {justification}")

# TODO: add an archival andidate tool (provide justification) 

read_resume_tool = client.create_tool(read_resume) 
submit_evaluation_tool = client.create_tool(submit_evaluation)

In [9]:
skills = "Front-end (React, Typescript), software engineering " \
+ "(ideally Python), and experience with LLMs."
eval_persona = f"You are responsible to finding good recruiting " \
+ "candidates, for the company description. " \
+ f"Ideal canddiates have skills: {skills}. " \
+ "Submit your candidate evaluation with the submit_evaluation tool. "

eval_agent = client.create_agent(
    name="eval_agent", 
    memory=OrgMemory(
        persona=eval_persona, 
        org_block=org_block,
    ), 
    tools=[read_resume_tool.name, submit_evaluation_tool.name]
)


#### Outreach agent 
This agent will email candidates with customized emails. Since sending emails is a bit complicated, we'll just pretend we sent an email by printing it in the tool call. 

In [10]:
def email_candidate(self, content: str): 
    """
    Send an email

    Args: 
        content (str): Content of the email 
    """
    print("Pretend to email:", content)
    return

email_candidate_tool = client.create_tool(email_candidate)

In [13]:
outreach_persona = "You are responsible for sending outbound emails " \
+ "on behalf of a company with the send_emails tool to " \
+ "potential candidates. " \
+ "If possible, make sure to personalize the email by appealing " \
+ "to the recipient with details about the company. " \
+ "You position is `Head Recruiter`, and you go by the name Bob, with contact info bob@gmail.com. " \
+ """
Follow this email template: 

Hi <candidate name>, 

<content> 

Best, 
<your name> 
<company name> 
"""

outreach_agent = client.create_agent(
    name="outreach_agent", 
    memory=OrgMemory(
        persona=outreach_persona, 
        org_block=org_block
    ), 
    tools=[email_candidate_tool.name]
)

Next, we'll send a message from the user telling the `leadgen_agent` to evaluate a given candidate: 

In [14]:
response = client.send_message(
    agent_name="eval_agent", 
    role="user", 
    message="Candidate: Tony Stark"
)

In [15]:
response

LettaResponse(messages=[InternalMonologue(id='message-eda4b3ed-c49b-46e0-a328-389e1a4f99f2', date=datetime.datetime(2024, 11, 7, 4, 9, 57, 382192, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='Checking candidate details for Tony Stark. Need to assess suitability for our roles.'), FunctionCallMessage(id='message-eda4b3ed-c49b-46e0-a328-389e1a4f99f2', date=datetime.datetime(2024, 11, 7, 4, 9, 57, 382192, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='read_resume', arguments='{\n  "name": "Tony Stark",\n  "request_heartbeat": true\n}', function_call_id='call_BkJmry1mIebLCRrMS0c5OyKh')), FunctionReturn(id='message-f25ef27a-8e64-489d-b106-c6cc8d7bfc91', date=datetime.datetime(2024, 11, 7, 4, 9, 57, 384351, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "Failed",\n  "message": "Error calling function read_resume: [Errno 2] No such file or directory: \'data/resumes/tony_stark.txt\'",\n  "time": "2024-11-06 08:09:57 PM PST-0800"\n}', status='error', function_call_id='call_BkJmry1mIebLCRrMS0c5OyKh'), InternalMonologue(id='message-8bd8537d-b07a-433e-8db2-cac21643b68b', date=datetime.datetime(2024, 11, 7, 4, 10, 0, 919112, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue="I couldn't retrieve the resume for Tony Stark. I'll need to communicate that back to the user and suggest another action."), FunctionCallMessage(id='message-8bd8537d-b07a-433e-8db2-cac21643b68b', date=datetime.datetime(2024, 11, 7, 4, 10, 0, 919112, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='send_message', arguments='{\n  "message": "It looks like I\'m having trouble accessing Tony Stark\'s resume right now. Could you provide any details you have on his skills or experience?"\n}', function_call_id='call_AiGCTzL94JmsURKnLKLANRXL')), FunctionReturn(id='message-c3a0f8fd-f894-46df-a091-1fcdf2cb7d4b', date=datetime.datetime(2024, 11, 7, 4, 10, 0, 919561, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-06 08:10:00 PM PST-0800"\n}', status='success', function_call_id='call_AiGCTzL94JmsURKnLKLANRXL')], usage=LettaUsageStatistics(completion_tokens=109, prompt_tokens=4997, total_tokens=5106, step_count=2))

#### Providing feedback to agents 
Since MemGPT agents are persisted, we can provide feedback to agents that is used in future agent executions if we want to modify the future behavior. 

In [16]:
feedback = "Our company pivoted to foundation model training"
response = client.send_message(
    agent_name="eval_agent", 
    role="user", 
    message=feedback
)

/Users/sarahwooders/repos/letta/letta/helpers/tool_rule_solver.py:70: UserWarning: User provided tool rules and execution state resolved to no more possible tool calls.
  warnings.warn(message)


In [17]:

feedback = "The company is also renamed to FoundationAI"
response = client.send_message(
    agent_name="eval_agent", 
    role="user", 
    message=feedback
)

In [19]:
response

LettaResponse(messages=[InternalMonologue(id='message-de6a4e9f-225a-4239-beba-f53bab429083', date=datetime.datetime(2024, 11, 7, 4, 10, 17, 338163, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='Updating core memory to reflect the new company name as FoundationAI. This is crucial information for ongoing evaluations and discussions.'), FunctionCallMessage(id='message-de6a4e9f-225a-4239-beba-f53bab429083', date=datetime.datetime(2024, 11, 7, 4, 10, 17, 338163, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='core_memory_replace', arguments='{\n  "label": "company",\n  "old_content": "The company has pivoted to foundation model training.",\n  "new_content": "The company is now named FoundationAI and has pivoted to foundation model training.",\n  "request_heartbeat": true\n}', function_call_id='call_QBMhZrxD0oBavqAMOn5nAV2g')), FunctionReturn(id='message-a24ee34b-24d3-4af0-8568-bec3d9bc9ac1', date=datetime.datetime(2024, 11, 7, 4, 10, 17, 338911, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-06 08:10:17 PM PST-0800"\n}', status='success', function_call_id='call_QBMhZrxD0oBavqAMOn5nAV2g'), InternalMonologue(id='message-55c94d9f-de07-4721-b2bb-e447314e7865', date=datetime.datetime(2024, 11, 7, 4, 10, 20, 546442, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='I need to confirm the updated company name and prompt for more information about Tony.'), FunctionCallMessage(id='message-55c94d9f-de07-4721-b2bb-e447314e7865', date=datetime.datetime(2024, 11, 7, 4, 10, 20, 546442, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='send_message', arguments='{\n  "message": "Got it! I\'ve updated the name to FoundationAI. Do you have any details on Tony Stark\'s skills that match our new focus?"\n}', function_call_id='call_KUJ9Id8yXdj4gt48C1mKlXUg')), FunctionReturn(id='message-c1ebe9cc-529e-407f-a01d-c43ffddee52b', date=datetime.datetime(2024, 11, 7, 4, 10, 20, 547869, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-06 08:10:20 PM PST-0800"\n}', status='success', function_call_id='call_KUJ9Id8yXdj4gt48C1mKlXUg')], usage=LettaUsageStatistics(completion_tokens=142, prompt_tokens=6387, total_tokens=6529, step_count=2))

In [20]:
response = client.send_message(
    agent_name="eval_agent", 
    role="system", 
    message="Candidate: Spongebob Squarepants"
)

In [21]:
client.get_core_memory(eval_agent.id).get_block("company")

Block(value='The company is called AgentOS and is building AI tools to make it easier to create and deploy LLM agents.\nThe company is now named FoundationAI and has pivoted to foundation model training.', limit=2000, template_name=None, template=False, label='company', description=None, metadata_={}, user_id=None, id='block-6db0fe1a-1f5e-44ab-852c-a2df8d7ab80e')

In [22]:
client.get_core_memory(outreach_agent.id).get_block("company")

Block(value='The company is called AgentOS and is building AI tools to make it easier to create and deploy LLM agents.', limit=2000, template_name=None, template=False, label='company', description=None, metadata_={}, user_id=None, id='block-6db0fe1a-1f5e-44ab-852c-a2df8d7ab80e')

## Section 3: Adding an orchestrator agent 
So far, we've been triggering the `eval_agent` manually. We can also create an additional agent that is responsible for orchestrating tasks. 

In [23]:
#re-create agents 
client.delete_agent(eval_agent.id)
client.delete_agent(outreach_agent.id)

eval_agent = client.create_agent(
    name="eval_agent", 
    memory=OrgMemory(
        persona=eval_persona, 
        org_block=org_block,
    ), 
    tools=[read_resume_tool.name, submit_evaluation_tool.name]
)

outreach_agent = client.create_agent(
    name="outreach_agent", 
    memory=OrgMemory(
        persona=outreach_persona, 
        org_block=org_block
    ), 
    tools=[email_candidate_tool.name]
)

The `recruiter_agent` will be linked to the same `org_block` that we created before - we can look up the current data in `org_block` by looking up its ID: 

In [24]:
client.get_block(org_block.id)

Block(value='The company is called AgentOS and is building AI tools to make it easier to create and deploy LLM agents.\nThe company is now named FoundationAI and has pivoted to foundation model training.', limit=2000, template_name=None, template=False, label='company', description=None, metadata_={}, user_id='user-00000000-0000-4000-8000-000000000000', id='block-6db0fe1a-1f5e-44ab-852c-a2df8d7ab80e')

In [25]:
from typing import Optional

def search_candidates_db(self, page: int) -> Optional[str]: 
    """
    Returns 1 candidates per page. 
    Page 0 returns the first 1 candidate, 
    Page 1 returns the next 1, etc.
    Returns `None` if no candidates remain. 

    Args: 
        page (int): The page number to return candidates from 

    Returns: 
        candidate_names (List[str]): Names of the candidates
    """
    
    names = ["Tony Stark", "Spongebob Squarepants", "Gautam Fang"]
    if page >= len(names): 
        return None
    return names[page]

def consider_candidate(self, name: str): 
    """
    Submit a candidate for consideration. 

    Args: 
        name (str): Candidate name to consider 
    """
    from letta import create_client 
    client = create_client()
    message = f"Consider candidate {name}" 
    print("Sending message to eval agent: ", message)
    response = client.send_message(
        agent_name="eval_agent", 
        role="user", 
        message=message
    ) 


# create tools 
search_candidate_tool = client.create_tool(search_candidates_db)
consider_candidate_tool = client.create_tool(consider_candidate)

# create recruiter agent
recruiter_agent = client.create_agent(
    name="recruiter_agent", 
    memory=OrgMemory(
        persona="You run a recruiting process for a company. " \
        + "Your job is to continue to pull candidates from the " 
        + "`search_candidates_db` tool until there are no more " \
        + "candidates left. " \
        + "For each candidate, consider the candidate by calling "
        + "the `consider_candidate` tool. " \
        + "You should continue to call `search_candidates_db` " \
        + "followed by `consider_candidate` until there are no more " \
        " candidates. ",
        org_block=org_block
    ), 
    tools=[search_candidate_tool.name, consider_candidate_tool.name]
)
    


In [26]:
response = client.send_message(
    agent_name="recruiter_agent", 
    role="system", 
    message="Run generation"
)

In [27]:
response

LettaResponse(messages=[InternalMonologue(id='message-5fee1cc7-b1f8-442d-a0cb-f291d361c4bd', date=datetime.datetime(2024, 11, 7, 4, 11, 4, 98419, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='User has logged in for the first time. Exciting!'), FunctionCallMessage(id='message-5fee1cc7-b1f8-442d-a0cb-f291d361c4bd', date=datetime.datetime(2024, 11, 7, 4, 11, 4, 98419, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='send_message', arguments='{\n  "message": "Welcome! It\'s great to have you here. Let\'s dive into your journey together, shall we?"\n}', function_call_id='call_LtMblVxNr2FVPiCJicAtaji9')), FunctionReturn(id='message-567a8bfb-468b-4292-82b8-c9ecf20ec7e9', date=datetime.datetime(2024, 11, 7, 4, 11, 4, 98638, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-06 08:11:04 PM PST-0800"\n}', status='success', function_call_id='call_LtMblVxNr2FVPiCJicAtaji9')], usage=LettaUsageStatistics(completion_tokens=50, prompt_tokens=2399, total_tokens=2449, step_count=1))

In [28]:
client.delete_agent(eval_agent.id)
client.delete_agent(outreach_agent.id)

In [29]:
client.delete_agent(recruiter_agent.id)

In [30]:
for agent in client.list_agents(): 
    client.delete_agent(agent.id)